In [1]:
from mmpose.datasets import build_dataloader, build_dataset
from mmcv import Config
from mmpose.models import build_posenet
from mmcv.runner import get_dist_info, init_dist, load_checkpoint
import mmcv
from mmcv.parallel import MMDataParallel
from mmpose.apis import multi_gpu_test, single_gpu_test

In [2]:
from mmpose.models import build_posenet

In [3]:
config_file = '/mnt/ssd/marley/ID_Card/mmpose/configs/ID_card/baseline_3_higherhrnet_w32_IDCard_640x640.py'
cfg = Config.fromfile(config_file)

In [10]:
cfg.model.pretrained = None
cfg.data.test.test_mode = True
eval_config = cfg.get('evaluation', {})

In [5]:
# build the dataloader
dataset = build_dataset(cfg.data.test, dict(test_mode=True))
dataloader_setting = dict(
    samples_per_gpu=16,
    workers_per_gpu=cfg.data.get('workers_per_gpu', 1),
    dist=False,
    shuffle=False,
    drop_last=False)
dataloader_setting = dict(dataloader_setting,
                              **cfg.data.get('test_dataloader', {}))
data_loader = build_dataloader(dataset, **dataloader_setting)

loading annotations into memory...
Done (t=0.01s)
=> num_images: 663


In [11]:
# # build the model and load checkpoint
model = build_posenet(cfg.model)
model = MMDataParallel(model, device_ids=[0])
work_dir = '/mnt/ssd/marley/ID_Card/mmpose/work_dirs/baseline_3_higherhrnet_640x640'
checkpoints = [os.path.join(work_dir, f) for f in os.listdir(work_dir)
               if f.endswith('.pth')]

In [13]:
for checkpoint in checkpoints:
    print('loading checkpoint from {}'.format(checkpoint))
    load_checkpoint(model, checkpoint, map_location='cpu')
    outputs = single_gpu_test(model, data_loader)
    results = dataset.evaluate(outputs, work_dir, **eval_config)
    print(results)

loading checkpoint from /mnt/ssd/marley/ID_Card/mmpose/work_dirs/baseline_3_higherhrnet_640x640/epoch_23.pth
Use load_from_local loader
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 663/663, 25.7 task/s, elapsed: 26s, ETA:     0sOrderedDict([('AP', 0.8463139110033002), ('AP .5', 0.9414787486430255), ('AP .75', 0.8257453008428166), ('AP (M)', -1.0), ('AP (L)', 0.8463139110033002), ('AR', 0.8675757575757576), ('AR .5', 0.953030303030303), ('AR .75', 0.8454545454545455), ('AR (M)', -1.0), ('AR (L)', 0.8675757575757576)])
loading checkpoint from /mnt/ssd/marley/ID_Card/mmpose/work_dirs/baseline_3_higherhrnet_640x640/epoch_50.pth
Use load_from_local loader
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 663/663, 25.7 task/s, elapsed: 26s, ETA:     0sOrderedDict([('AP', 0.8493118286053308), ('AP .5', 0.932427349555103), ('AP .75', 0.8484927328718777), ('AP (M)', -1.0), ('AP (L)', 0.8493118286053308), ('AR', 0.8728915662650601), ('AR .5', 0.9487951807228916), ('AR .75', 0.8629518072289156), ('AR (M)', -1.0), ('AR (L)', 0.

In [7]:
import json

In [8]:
eval_config = cfg.get('evaluation', {})
with open ('/mnt/ssd/marley/ID_Card/mmpose/work_dirs/baseline_debug/results_138.json') as f:
    outputs = json.load(f)
print(type(outputs))

<class 'list'>


In [6]:
work_dir = '/mnt/ssd/marley/ID_Card/mmpose/work_dirs/overfit_higherhrnet_w32_IDCard_512x512'
results = dataset.evaluate(outputs, work_dir, **eval_config)

NameError: name 'outputs' is not defined